In [87]:
import numpy as np
import pandas as pd
import re
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import StratifiedKFold,cross_val_predict
from keras.wrappers.scikit_learn import KerasClassifier


In [2]:
work_train= {} 
work_test = {}
pre_process=["tfidf_tsvd_100.csv","w2v_300.csv","bio.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [10]:
#External features + genes dummies
feat_train_1=pd.read_csv("w_meta_features/meta_train_l1l2.csv") #Gene dummies
feat_test_1=pd.read_csv("w_meta_features/meta_test_l1l2.csv") 
feat_train_1=feat_train_1.drop("ID",axis=1)
feat_test_1=feat_test_1.drop("ID",axis=1)
feat_train_2=pd.read_csv("w_meta_features/new_working_train_ext.csv")#External tsvd
feat_test_2=pd.read_csv("w_meta_features/new_working_test_ext.csv")

In [14]:
train=pd.concat((work_train["bio"],work_train["tfidf_tsvd_100"],work_train["w2v_300"],feat_train_1,feat_train_2),axis=1)

In [15]:
test=pd.concat((work_test["bio"],work_test["tfidf_tsvd_100"],work_test["w2v_300"],feat_test_1,feat_test_2),axis=1)

In [17]:
print(train.shape)
print(test.shape)

(2925, 670)
(619, 670)


In [65]:
new_train=pd.read_csv("checkpoints_databases/new_working_train.csv")
new_test=pd.read_csv("checkpoints_databases/new_working_test.csv")
y=new_train.Class - 1
ID_train=new_train.ID
ID_test=new_test.ID
del new_train, new_test

In [27]:
y_cat=to_categorical(y,9)

In [68]:
kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [90]:
np.random.seed(26)
def create_model():
    model=Sequential()
    model.add(Dense(30, input_dim=670, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(30,activation="relu"))
    model.add(Dense(9, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [93]:
model = KerasClassifier(build_fn=create_model,epochs=80, batch_size=30,validation_split=0.2,callbacks=[es,checkpoint],verbose=0)
es=EarlyStopping(monitor="val_loss",mode="min",patience=10)
checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="val_loss",mode="min")

In [102]:
def model_gen_neuralnet(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    es=EarlyStopping(monitor="val_loss",mode="min",patience=10)
    checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="val_loss",mode="min")
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            
            clf.fit(X_train,to_categorical(y_train,9),epochs=80,batch_size=30,validation_split=0.2,callbacks=[es,checkpoint],verbose=0)
            clf.load_weights("checknn")
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5

    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("new_scores/new_stack_test/new_stem_{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    model = KerasClassifier(build_fn=create_model,epochs=80, batch_size=30,callbacks=[es,checkpoint],verbose=0)
    y_pred=cross_val_predict(estimator=model,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("new_scores/new_stack_train/new_stem_{}.csv".format(file),index=False)

In [104]:

model_gen_neuralnet(X=np.array(train),X_test=np.array(test),y=y,classifier=create_model(),file="neuralnet")


Fold 1 (2337, 670) (588, 670)
 32/619 [>.............................] - ETA: 0scross_val sur train 


TypeError: can't pickle _thread.lock objects